In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_rows=500

In [2]:
cd ..

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus


In [3]:
from hopkins_cleaner import HopkinsDataCleaner

## Create instances for each URL 

In [4]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'

In [5]:
confirmed = HopkinsDataCleaner(url=url_confirmed,
                              dataset_name='cases')
recovered = HopkinsDataCleaner(url=url_recovered,
                              dataset_name='recoveries')
deaths = HopkinsDataCleaner(url=url_deaths,
                           dataset_name='deaths')

## Execute each instance and save the `.data` attribute 

In [6]:
confirmed.run()
recovered.run()
deaths.run()

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/us_data_cleaner.py:138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  axis=0)


In [7]:
df_main = confirmed.data
df_recovered = recovered.data
df_deaths = deaths.data

In [8]:
df_main.head(2)

,confirmed_cases,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases
0,2,Thailand,2020-01-22,15.0,101.0,Not Provided,Not Provided-Thailand,1.0,0.0,2.0
1,3,Thailand,2020-01-23,15.0,101.0,Not Provided,Not Provided-Thailand,2.0,2.0,1.0


In [9]:
df_recovered.head(2)

,confirmed_recoveries,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_recoveries_prev_day,daily_diff_recoveries
0,0,Thailand,2020-01-22,15.0,101.0,Not Provided,Not Provided-Thailand,1.0,0.0,0.0
1,0,Thailand,2020-01-23,15.0,101.0,Not Provided,Not Provided-Thailand,2.0,0.0,0.0


## Trim non-base datasets 

In [10]:
df_recovered = df_recovered[['confirmed_recoveries', 
              'date', 
              'state_and_country', 
              'confirmed_recoveries_prev_day', 
              'daily_diff_recoveries']]
df_deaths = df_deaths[['confirmed_deaths', 
              'date', 
              'state_and_country', 
              'confirmed_deaths_prev_day', 
              'daily_diff_deaths']]

In [11]:
df_main.shape

(12012, 10)

In [12]:
df_final = df_main.merge(df_recovered,
             how='left',
             on=['date','state_and_country'])

In [13]:
df_final = df_final.merge(df_deaths,
             how='left',
             on=['date','state_and_country'])

In [14]:
df_final.shape

(12012, 16)

In [15]:
ref_table = pd.read_csv('ref_table_us_states.csv')
ref_table.columns = ref_table.columns.str.lower().str.replace(' ','_')

In [16]:
df_final = df_final.merge(ref_table,
             how='left',
             left_on='province_state',
             right_on='state_code')

In [17]:
df_final['new_lat'] = df_final['lat'].combine_first(df_final['latitude'])

In [18]:
df_final['new_long'] = df_final['long'].combine_first(df_final['longitude'])

In [19]:
df_final.head(2)

,confirmed_cases,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,...,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_code,state_name,latitude,longitude,new_lat,new_long
0,2,Thailand,2020-01-22,15.0,101.0,Not Provided,Not Provided-Thailand,1.0,0.0,2.0,...,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,15.0,101.0
1,3,Thailand,2020-01-23,15.0,101.0,Not Provided,Not Provided-Thailand,2.0,2.0,1.0,...,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,15.0,101.0


In [20]:
df_final = df_final.drop(labels=['long','lat','latitude','longitude','state_code'], axis=1)

In [21]:
df_final = df_final.rename(columns={'new_long':'long',
                       'new_lat':'lat'})

In [22]:
df_final.shape

(12012, 17)

In [23]:
df_final

,confirmed_cases,country_region,date,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_name,lat,long
0,2,Thailand,2020-01-22,Not Provided,Not Provided-Thailand,1.0,0.0,2.0,0,0.0,0.0,0,0.0,0.0,NaN,15.00000,101.000000
1,3,Thailand,2020-01-23,Not Provided,Not Provided-Thailand,2.0,2.0,1.0,0,0.0,0.0,0,0.0,0.0,NaN,15.00000,101.000000
2,5,Thailand,2020-01-24,Not Provided,Not Provided-Thailand,3.0,3.0,2.0,0,0.0,0.0,0,0.0,0.0,NaN,15.00000,101.000000
3,7,Thailand,2020-01-25,Not Provided,Not Provided-Thailand,4.0,5.0,2.0,0,0.0,0.0,0,0.0,0.0,NaN,15.00000,101.000000
4,8,Thailand,2020-01-26,Not Provided,Not Provided-Thailand,5.0,7.0,1.0,2,0.0,2.0,0,0.0,0.0,NaN,15.00000,101.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12007,0,US,2020-03-13,WV,WV-US,52.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,West Virginia,39.00000,-80.500000
12008,0,US,2020-03-10,WY,WY-US,49.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,Wyoming,43.07597,-107.290283
12009,0,US,2020-03-11,WY,WY-US,50.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,Wyoming,43.07597,-107.290283
12010,1,US,2020-03-12,WY,WY-US,51.0,0.0,1.0,0,0.0,0.0,0,0.0,0.0,Wyoming,43.07597,-107.290283


In [24]:
max_date = df_final['date'].max()

In [25]:
yesterday = df_final.loc[df_final['date'] == max_date]
yesterday['confirmed_cases'].sum(), yesterday['confirmed_recoveries'].sum(), yesterday['confirmed_deaths'].sum()

(145193, 70251, 5404)

In [26]:
df_final['daily_diff_cases'].sum(), df_final['daily_diff_recoveries'].sum(), df_final['daily_diff_deaths'].sum()

(145193.0, 70251.0, 5404.0)

In [27]:
df_final.loc[df_final['province_state'] == 'WA'].tail(5)

,confirmed_cases,country_region,date,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_name,lat,long
11651,122,US,2020-03-09,WA,WA-US,48.0,122.0,0.0,1,1.0,0.0,19,18.0,1.0,Washington,47.751076,-120.740135
11996,267,US,2020-03-10,WA,WA-US,49.0,122.0,145.0,1,1.0,0.0,23,19.0,4.0,Washington,47.751076,-120.740135
11997,366,US,2020-03-11,WA,WA-US,50.0,267.0,99.0,1,1.0,0.0,29,23.0,6.0,Washington,47.751076,-120.740135
11998,442,US,2020-03-12,WA,WA-US,51.0,366.0,76.0,1,1.0,0.0,31,29.0,2.0,Washington,47.751076,-120.740135
11999,568,US,2020-03-13,WA,WA-US,52.0,442.0,126.0,1,1.0,0.0,37,31.0,6.0,Washington,47.751076,-120.740135


In [28]:
df_final.loc[df_final['province_state'] == 'VA'].tail(5)

,confirmed_cases,country_region,date,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_name,lat,long
11555,2,US,2020-03-09,VA,VA-US,48.0,2.0,0.0,0,0.0,0.0,0,0.0,0.0,Virginia,37.926868,-78.024902
11988,7,US,2020-03-10,VA,VA-US,49.0,2.0,5.0,0,0.0,0.0,0,0.0,0.0,Virginia,37.926868,-78.024902
11989,9,US,2020-03-11,VA,VA-US,50.0,7.0,2.0,0,0.0,0.0,0,0.0,0.0,Virginia,37.926868,-78.024902
11990,17,US,2020-03-12,VA,VA-US,51.0,9.0,8.0,0,0.0,0.0,0,0.0,0.0,Virginia,37.926868,-78.024902
11991,30,US,2020-03-13,VA,VA-US,52.0,17.0,13.0,0,0.0,0.0,0,0.0,0.0,Virginia,37.926868,-78.024902


In [29]:
df_final.loc[df_final['province_state'] == 'CA'].tail(5)

,confirmed_cases,country_region,date,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_name,lat,long
9443,101,US,2020-03-09,CA,CA-US,48.0,95.0,6.0,2,2.0,0.0,1,1.0,0.0,California,36.778259,-119.417931
11812,144,US,2020-03-10,CA,CA-US,49.0,101.0,43.0,2,2.0,0.0,2,1.0,1.0,California,36.778259,-119.417931
11813,177,US,2020-03-11,CA,CA-US,50.0,144.0,33.0,2,2.0,0.0,3,2.0,1.0,California,36.778259,-119.417931
11814,221,US,2020-03-12,CA,CA-US,51.0,177.0,44.0,6,2.0,4.0,4,3.0,1.0,California,36.778259,-119.417931
11815,282,US,2020-03-13,CA,CA-US,52.0,221.0,61.0,6,6.0,0.0,4,4.0,0.0,California,36.778259,-119.417931


In [30]:
df_final.to_csv('HOPKINS_CLEANED.csv', index=False)

## QA of Data 

In [37]:
# This indicates a shift in the data
df_final.loc[df_final['daily_diff_cases'] < 0]['date'].value_counts()

2020-03-06    2
2020-01-31    1
2020-01-23    1
2020-02-29    1
2020-02-07    1
2020-03-13    1
2020-02-02    1
2020-03-09    1
2020-03-10    1
Name: date, dtype: int64

In [38]:
# This indicates a shift in the data
df_final.loc[df_final['daily_diff_cases'] < 0]['province_state'].value_counts()

Not Provided             3
Queensland               2
OR                       1
Northern Territory       1
From Diamond Princess    1
Saint Barthelemy         1
Diamond Princess         1
Name: province_state, dtype: int64

In [39]:
df_final.loc[df_final['daily_diff_cases'] < 0]

,confirmed_cases,country_region,date,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_name,lat,long
53,1,Japan,2020-01-23,Not Provided,Not Provided-Japan,2.0,2.0,-1.0,0,0.0,0.0,0,0.0,0.0,NaN,36.0000,138.0000
68,25,Japan,2020-02-07,Not Provided,Not Provided-Japan,17.0,45.0,-20.0,1,1.0,0.0,0,0.0,0.0,NaN,36.0000,138.0000
425,2,Australia,2020-01-31,Queensland,Queensland-Australia,10.0,3.0,-1.0,0,0.0,0.0,0,0.0,0.0,NaN,-28.0167,153.4000
427,2,Australia,2020-02-02,Queensland,Queensland-Australia,12.0,3.0,-1.0,0,0.0,0.0,0,0.0,0.0,NaN,-28.0167,153.4000
1182,0,Australia,2020-02-29,From Diamond Princess,From Diamond Princess-Australia,39.0,8.0,-8.0,0,0.0,0.0,0,0.0,0.0,NaN,35.4437,139.6380
1455,189,Bahrain,2020-03-13,Not Provided,Not Provided-Bahrain,52.0,195.0,-6.0,44,35.0,9.0,0,0.0,0.0,NaN,26.0275,50.5500
3788,0,Australia,2020-03-06,Northern Territory,Northern Territory-Australia,45.0,1.0,-1.0,0,0.0,0.0,0,0.0,0.0,NaN,-12.4634,130.8456
6024,696,Cruise Ship,2020-03-06,Diamond Princess,Diamond Princess-Cruise Ship,45.0,706.0,-10.0,40,10.0,30.0,6,6.0,0.0,NaN,35.4437,139.6380
8107,1,France,2020-03-09,Saint Barthelemy,Saint Barthelemy-France,48.0,3.0,-2.0,0,0.0,0.0,0,0.0,0.0,NaN,17.9000,-62.8333
11956,0,US,2020-03-10,OR,OR -US,49.0,2.0,-2.0,0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN


In [40]:
df_final.loc[(df_final['daily_diff_cases'] < 0) &
      (df_final['province_state'] == 'CA')]

,confirmed_cases,country_region,date,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_name,lat,long


In [41]:
df_final.loc[df_final['country_region'] == 'China'].head(200).sort_values('rank')

,confirmed_cases,country_region,date,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths,state_name,lat,long
5408,444,China,2020-01-22,Hubei,Hubei-China,1.0,0.0,444.0,28,0.0,28.0,17,0.0,17.0,NaN,30.9756,112.2707
5720,10,China,2020-01-22,Zhejiang,Zhejiang-China,1.0,0.0,10.0,0,0.0,0.0,0,0.0,0.0,NaN,29.1832,120.0934
5616,26,China,2020-01-22,Guangdong,Guangdong-China,1.0,0.0,26.0,0,0.0,0.0,0,0.0,0.0,NaN,23.3417,113.4244
5668,5,China,2020-01-22,Henan,Henan-China,1.0,0.0,5.0,0,0.0,0.0,0,0.0,0.0,NaN,33.8820,113.6140
5409,444,China,2020-01-23,Hubei,Hubei-China,2.0,444.0,0.0,28,28.0,0.0,17,17.0,0.0,NaN,30.9756,112.2707
5721,27,China,2020-01-23,Zhejiang,Zhejiang-China,2.0,10.0,17.0,0,0.0,0.0,0,0.0,0.0,NaN,29.1832,120.0934
5617,32,China,2020-01-23,Guangdong,Guangdong-China,2.0,26.0,6.0,2,0.0,2.0,0,0.0,0.0,NaN,23.3417,113.4244
5669,5,China,2020-01-23,Henan,Henan-China,2.0,5.0,0.0,0,0.0,0.0,0,0.0,0.0,NaN,33.8820,113.6140
5670,9,China,2020-01-24,Henan,Henan-China,3.0,5.0,4.0,0,0.0,0.0,0,0.0,0.0,NaN,33.8820,113.6140
5722,43,China,2020-01-24,Zhejiang,Zhejiang-China,3.0,27.0,16.0,1,0.0,1.0,0,0.0,0.0,NaN,29.1832,120.0934


In [42]:
df_final.country_region.value_counts()

US                        2808
China                     1716
Australia                  468
Canada                     416
France                     208
United Kingdom             156
Denmark                    104
Reunion                     52
Argentina                   52
Jamaica                     52
Dominican Republic          52
Azerbaijan                  52
Poland                      52
Sweden                      52
Nigeria                     52
Kuwait                      52
Czechia                     52
United Arab Emirates        52
Vietnam                     52
Pakistan                    52
Tunisia                     52
Armenia                     52
Germany                     52
Norway                      52
Bahrain                     52
Finland                     52
Qatar                       52
Korea, South                52
Austria                     52
Aruba                       52
Cayman Islands              52
Singapore                   52
North Ma